In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas

In [2]:
spark = SparkSession.builder.appName("test_session").getOrCreate()
spark

24/01/08 15:47:48 WARN Utils: Your hostname, arthurpfonseca resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlo1)
24/01/08 15:47:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/08 15:47:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_spark = spark.read.csv("bla.csv", inferSchema=True)

In [4]:
df_spark.show()

+----+----+------+
| _c0| _c1|   _c2|
+----+----+------+
|bla1|bla2|  name|
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
|NULL|NULL|  NULL|
+----+----+------+



In [5]:
df_spark = spark.read.option("header", "true").csv("bla.csv", inferSchema=True)

In [6]:
df_spark.printSchema()

root
 |-- bla1: integer (nullable = true)
 |-- bla2: integer (nullable = true)
 |-- name: string (nullable = true)



In [7]:
df_spark.show()

+----+----+------+
|bla1|bla2|  name|
+----+----+------+
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
|NULL|NULL|  NULL|
+----+----+------+



In [8]:
df_spark.select(["bla1", "bla2"]).show()

+----+----+
|bla1|bla2|
+----+----+
|   1|   2|
|   3|   4|
|   5|   6|
|NULL|NULL|
+----+----+



In [9]:
df_spark["bla1"], df_spark.dtypes

(Column<'bla1'>, [('bla1', 'int'), ('bla2', 'int'), ('name', 'string')])

In [10]:
df_spark = df_spark.withColumn("col+2", df_spark["bla2"] + 2)
df_spark.show()

+----+----+------+-----+
|bla1|bla2|  name|col+2|
+----+----+------+-----+
|   1|   2|arthur|    4|
|   3|   4|arthur|    6|
|   5|   6|felype|    8|
|NULL|NULL|  NULL| NULL|
+----+----+------+-----+



In [11]:
df_spark = df_spark.drop("col+2")
df_spark.show()

+----+----+------+
|bla1|bla2|  name|
+----+----+------+
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
|NULL|NULL|  NULL|
+----+----+------+



In [12]:
df_spark.withColumnRenamed("bla2", "bla3").show()

+----+----+------+
|bla1|bla3|  name|
+----+----+------+
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
|NULL|NULL|  NULL|
+----+----+------+



In [13]:
df_spark.na.drop(how="any", thresh=0).show()

+----+----+------+
|bla1|bla2|  name|
+----+----+------+
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
|NULL|NULL|  NULL|
+----+----+------+



In [14]:
df_spark.na.drop(how="any", subset=["bla1"]).show()

+----+----+------+
|bla1|bla2|  name|
+----+----+------+
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
+----+----+------+



In [15]:
df_spark.na.fill(123, subset=["bla1"]).show()

+----+----+------+
|bla1|bla2|  name|
+----+----+------+
|   1|   2|arthur|
|   3|   4|arthur|
|   5|   6|felype|
| 123|NULL|  NULL|
+----+----+------+



In [16]:
df_spark.filter("bla1>=3").show()

+----+----+------+
|bla1|bla2|  name|
+----+----+------+
|   3|   4|arthur|
|   5|   6|felype|
+----+----+------+



In [19]:
df_spark.groupBy("name").sum().show()
df_spark.groupBy("name").count().show()

+------+---------+---------+
|  name|sum(bla1)|sum(bla2)|
+------+---------+---------+
|felype|        5|        6|
|  NULL|     NULL|     NULL|
|arthur|        4|        6|
+------+---------+---------+

+------+-----+
|  name|count|
+------+-----+
|felype|    1|
|  NULL|    1|
|arthur|    2|
+------+-----+



In [24]:
from pyspark.ml.feature import VectorAssembler

feature_assembler = VectorAssembler(inputCols=["bla1", "bla2"], outputCol="independent_features")
feature_assembler.transform(df_spark)

DataFrame[bla1: int, bla2: int, name: string, independent_features: vector]